# Rapport projet FAT (Barreaux Alexis - Belfer Adrien)


## Modélisation

Cette partie contient simplement les choix fait en terme de modélisation et le calcul des $\alpha$, elle peut donc être passée au besoin.

### Modèle utilisé

On suppose dans le modèle des colonies que le taux de passage de l'état $n$ à l'état $T_{jk}(n)$ (ie un vélo est passé de la colonie j à la colonie k), notée $q(n,T_{jk}(n))$, vérifie $q(n,T_{jk}(n)) = \lambda_{jk}\phi_j(n_j), \phi_j(0)=0$

De plus, on a bien ici que le système est irréductible car on peut aller de toute station (a fortiori colonie) à toute autre en un nombre fini d'étapes. Donc on se trouve bien dans un processus fermé de migration.

Si on est dans une colonie qui correspond à une route, on est forcément "servi" et tout se passe comme si on avait un nombre infini de serveurs. Si on est dans une station par contre, on ne peut être "servi" que si il y a un vélo en place. Dans toute la suite, les 5 premières colonies correspondent aux stations et les 20 suivantes aux routes. On a donc :

$\phi_j(n) = n_j$ si $j \geq 6, \displaystyle{1\!\!1_{\{n_j\geq1\}}}$ sinon.

De même, si on est dans une colonie j qui correspond à une route, $\lambda_{jk}$ est nul si k n'est pas la destination de la route et sinon correspond à l'inverse du temps moyen de parcours. C'est-à-dire, si on est dans la colonie qui correspond à la route $1 \rightarrow 2$ par exemple, $\lambda_{jk} = 0, k \ne2$ et $\lambda_{jk} = 60 *\frac{1}{3}$ sinon, où le 60 intervient pour être homogène en heures.

Si par contre $j \leq 5$, on se trouve dans une colonie et $\lambda_{jk}$ est nul si k n'est pas une route partant de cette colonie, et vaut la probabilité de choisir le chemin $j \rightarrow k$ multiplié par le taux de départ de j. Soit par exemple si on se trouve dans la station 1 et qu'on part vers 2: $\lambda_{jk}= 0.22 * 2.8$.

D'où au final:
- si $(j,k) \not\in $ $$\{(1, 6), (1, 7), (1, 8), (1, 9), (2, 10), (2, 11), (2, 12), (2, 13), (3, 14), (3, 15), \\
(3, 16), (3, 17),(4, 18), (4, 19), (4, 20), (4, 21), (5, 22), (5, 23), (5, 24), (5, 25) \} \\ 
\cup\{ (6, 2), (7, 3), (8, 4), (9, 5), (10, 1), (11, 3), (12, 4), (13, 5), (14, 1), (15, 2), (16, 4),\\
 (17, 5), (18, 1), (19, 2), (20, 3), (21, 5), (22, 1), (23, 2), (24, 3), (25, 4)\}$$
$q(n,T_{jk}(n))=0$
- sinon et si $j \leq 5$: $q(n,T_{jk}(n))= p_{j,k}*\lambda_j* \displaystyle{1\!\!1_{\{n_j\geq1\}}}$, où $p_{j,k}$ la probabilité en étant dans la colonie/station j de choisir le trajet associé à la colonie k (voir la matrice de routage) et $\lambda_j$ le taux de départ de la station.
- sinon et si $j \geq 6$: $q(n,T_{jk}(n))= 60 * \frac{1}{t_j} * n_j$, où $t_j$ est le temps moyen du trajet associé à la colonie en minutes.

### Loi stationnaire

#### Calcul des $\lambda_{j,k}$

In [40]:
def source_station_of_route_colony_i(i:int):
    return ((i-6)//4) + 1

# Data definition
valid_values_j_k = [
    (1, 6),
    (1, 7),
    (1, 8),
    (1, 9),
    (2, 10),
    (2, 11),
    (2, 12),
    (2, 13),
    (3, 14),
    (3, 15),
    (3, 16),
    (3, 17),
    (4, 18),
    (4, 19),
    (4, 20),
    (4, 21),
    (5, 22),
    (5, 23),
    (5, 24),
    (5, 25),
    (6, 2),
    (7, 3),
    (8, 4),
    (9, 5),
    (10, 1),
    (11, 3),
    (12, 4),
    (13, 5),
    (14, 1),
    (15, 2),
    (16, 4),
    (17, 5),
    (18, 1),
    (19, 2),
    (20, 3),
    (21, 5),
    (22, 1),
    (23, 2),
    (24, 3),
    (25, 4),
]
departure_average = [2.8, 3.7, 5.5, 3.5, 4.6]
routing = [
    [0, 0.22, 0.32, 0.2, 0.26],
    [0.17, 0, 0.34, 0.21, 0.28],
    [0.19, 0.26, 0, 0.24, 0.31],
    [0.17, 0.22, 0.33, 0, 0.28],
    [0.18, 0.24, 0.35, 0.23, 0],
]
mean_travel_times = [
    [0, 3, 5, 7, 7],
    [2, 0, 2, 5, 5],
    [4, 2, 0, 3, 3],
    [8, 6, 4, 0, 2],
    [7, 7, 5, 2, 0],
]

# Measuring lambdas
lambdas_j_k = dict()
for (j,k) in valid_values_j_k:
    # Lambda_j_k for stations
    if j <= 5:
        source_station = j
        source_station_index = j - 1
        
        road_from_source_number = ((k - 6) % 4) + 1

        destination_station = road_from_source_number if road_from_source_number < source_station else road_from_source_number + 1
        destination_station_index = destination_station - 1

        # print(j,k,source_station , destination_station, road_from_source_number)
        lambdas_j_k[(j,k)]= routing[source_station_index][destination_station_index] * departure_average[source_station_index]

    else:
        source_station = source_station_of_route_colony_i(j)
        source_station_index = source_station - 1

        destination_station = k
        destination_station_index = destination_station - 1

        #print(j,k,source_station , destination_station)

        lambdas_j_k[(j,k)] = 60 / mean_travel_times[source_station_index][destination_station_index]


print(lambdas_j_k)

{(1, 6): 0.616, (1, 7): 0.8959999999999999, (1, 8): 0.5599999999999999, (1, 9): 0.728, (2, 10): 0.6290000000000001, (2, 11): 1.2580000000000002, (2, 12): 0.777, (2, 13): 1.0360000000000003, (3, 14): 1.045, (3, 15): 1.4300000000000002, (3, 16): 1.3199999999999998, (3, 17): 1.705, (4, 18): 0.5950000000000001, (4, 19): 0.77, (4, 20): 1.155, (4, 21): 0.9800000000000001, (5, 22): 0.828, (5, 23): 1.1039999999999999, (5, 24): 1.6099999999999999, (5, 25): 1.058, (6, 2): 20.0, (7, 3): 12.0, (8, 4): 8.571428571428571, (9, 5): 8.571428571428571, (10, 1): 30.0, (11, 3): 30.0, (12, 4): 12.0, (13, 5): 12.0, (14, 1): 15.0, (15, 2): 30.0, (16, 4): 20.0, (17, 5): 20.0, (18, 1): 7.5, (19, 2): 10.0, (20, 3): 15.0, (21, 5): 30.0, (22, 1): 8.571428571428571, (23, 2): 8.571428571428571, (24, 3): 12.0, (25, 4): 30.0}


#### Calcul des $\alpha_j$

In [41]:
def routes_from_station_i_colony_numbers(i : int):
    return [6 + 4*(i-1) + j for j in range(4)]

print(routes_from_station_i_colony_numbers(3))

def routes_to_station_i_colony_numbers(i : int):
    routes = []
    for source_station in range(1,6):
        if source_station == i:
            continue
        else:
            road_number = i if i < source_station else i - 1
            routes.append(6 + 4*(source_station - 1) + road_number - 1)
    return routes


print(routes_to_station_i_colony_numbers(3))

[14, 15, 16, 17]
[7, 11, 20, 24]


In [42]:
import numpy as np

number_of_equations = 25

M = np.zeros((number_of_equations,25))
X = np.zeros((number_of_equations,1))
X[0] = 1
M[0] = np.ones(25)

for line_of_M in range(1,number_of_equations):
    j = line_of_M

    if j <= 5:
        source_station = j
        source_station_index = source_station - 1

        colonies_from_source = routes_from_station_i_colony_numbers(i=source_station)
        colonies_to_source = routes_to_station_i_colony_numbers(i=source_station)

        M[line_of_M][source_station_index]= np.sum([lambdas_j_k[(source_station, route_colony)] for route_colony in colonies_from_source])
        for colony_to_source in colonies_to_source:
            M[line_of_M][colony_to_source-1] = - lambdas_j_k[(colony_to_source,j)]

    else:
        source_station = ((j-6)//4) + 1
        source_station_index = source_station - 1

        road_from_source_number = ((j - 6) % 4) + 1

        destination_station = road_from_source_number if road_from_source_number < source_station else road_from_source_number + 1
        destination_station_index = destination_station - 1

        #print(j,k, source_station, destination_station, source_station_index, destination_station_index)
    
        M[line_of_M][j - 1] = lambdas_j_k[(j,destination_station)]
        M[line_of_M][source_station_index] = - lambdas_j_k[(source_station,j)]

        

#print(M[0:24:5, :], X.T)

In [43]:
#Résolution de M^{-1}X
import numpy as np

alphas = np.dot(np.linalg.inv(M),X)

#### Loi stationnaire

Ici comme les $\alpha$ somment à 1, on a $G_N=G_N^{-1}=1$ et $\pi(e_j)=\alpha_j$.

## Calcul théorique

### (1) Equations de trafic

On sait que l'on cherche une loi stationnaire sous la forme:

$\displaystyle\pi(n)=G_n^{-1}*\prod_{j=1}^{25}\frac{\alpha_j^{n_j}}{\displaystyle\prod_{r=1}^{n_j}\phi_j(r)}$

Donc pour $j \geq 6$ on a un terme en $\frac{\alpha_j^{n_j}}{n_j!}$ et pour $j \leq 5$ un terme $\alpha_j^{n_j}$ car alors $\displaystyle\prod_{r=1}^{n_j}\phi_j(r) = \prod_{r=1}^{n_j}\displaystyle{1\!\!1_{\{r\geq1\}}} = 1$.

Soit $\displaystyle\pi(n)=G_n^{-1}*\prod_{j=1}^{5}\alpha_j^{n_j}*\prod_{j=6}^{25}\frac{\alpha_j^{n_j}}{n_j!}$

Où, par les équations de balance locale sur $\pi$, les $\alpha_j$ vérifie les équations de trafic : 
$\alpha_j\displaystyle\sum_k\lambda_{j,k} =\sum_k\alpha_k\lambda_{k,j} $, avec $\alpha_j>0$ et $\displaystyle\sum_j\alpha_j=1$

Ce qui nous donne, en retirant les $\lambda$ nuls:
- si $j\leq 5$: $\alpha_j \sum_{k \in R_j^+}\lambda_{j, k}= \sum_{k \in R_j^-}\alpha_{k} * \lambda_{k, j}$ où $R_j^+$ et $R_j^-$ sont respectivement les indices des routes dont j est le départ et celles dont j est l'arrivée.
- si $j\geq 6$: $\alpha_j * \lambda_{j, k_{dest}}= \alpha_{k_{départ}} * \lambda_{k_{départ}, j} $ où $k_{dest}$ est l'unique colonie destination de cette route et $k_{départ}$ est l'unique colonie de départ de cette route.

Soit par exemple pour:
- j = 1: $\alpha_1 * (\lambda_{1, 6} + \lambda_{1, 7} + \lambda_{1, 8} + \lambda_{1, 9})= \alpha_{10} * \lambda_{10, 1} + \alpha_{14} * \lambda_{14, 1} + \alpha_{18} * \lambda_{18, 1} + \alpha_{22} * \lambda_{22, 1}$
- j = 6: $\alpha_6 * \lambda_{6, 2}= \alpha_1 * \lambda_{1, 6} $

### (2) Espace d'état

Si on a un seul vélo, l'espace d'état S est réduit à :
$S=\{ n\in \{0,1\}^J, \sum_{j=1}^J n_j = 1\}$,où J est le nombre total de colonies.

Soit ici J = nombre de stations + nombre de routes, ie $J= 5 + 5(5-1) = 25$

Donc on est réduit à choisir dans quelle colonie se trouve l'unique vélo $|S| = |J| = 25$.

### (3) Probabilité de stations vides

On note $e_i=(0,...,0,1,0,...0)$ ie le vecteur de taille J avec que des 0 sauf la i-ème coordonnée qui vaut 1.

Cette probabilité, notée $P_{vide}$ et qui équivaut au fait de perdre un client souhaitant utilisé un vélo, vaut alors:

$\displaystyle P_{vide}=P(n_1=n_2=n_3=n_4=n_5=0)=\sum_{n\in S, \sum_{j=6}^J n_j = 1}\pi(n)=\sum_{j=6}^{25} \pi(e_j)$

Où encore $\displaystyle P_{vide}=G_n^{-1}*(\sum_{j=6}^{25} \frac{\alpha_j^{1}}{1!})=G_n^{-1}*(\sum_{j=6}^{25} \alpha_j)$

In [50]:
# Cette probabilité a donc pour valeur ici (en prenant en compte le décalage inhérent au fait que les tableaux
# commencent à l'indice 0):

P_vide = np.sum(alphas[5:])
print("Probabilité que toutes les stations soient vides: ", round(P_vide,4), "à E-4 près.")

Probabilité que toutes les stations soient vides:  0.2177 à E-4 près.


### (4) Simulation

On a donc en théorie 21,77% de temps où toutes les station sont vides avec un seul vélo. Par la simulation on trouve pour 150 heures la valeur de 20,9%. En poussant à 150 000 heures on obtient toutefois 21,73%.

Remarque: les probabilités que les différentes stations soient vides sont respectivement (valeur théorique / valeur simulée sur 150 heures):
- station 1: 83,22% / 84,30%
- station 2: 83,89% / 86,17%
- station 3: 85,80% / 83,52%
- station 4: 83,85% / 83,77%
- station 5: 85,01% / 84,41%